In [0]:
# Parameterized values for catalog_name, schema_name, table_name
dbutils.widgets.text("catalog_name", "")
dbutils.widgets.text("schema_name", "")
dbutils.widgets.text("table_name", "")

# Get the parameterized values
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
name = dbutils.widgets.get("table_name")


In [0]:
import json
from datetime import datetime

# Function to convert datetime objects to strings
def convert_datetime(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError("Type not serializable")

# Retrieving all details from schema_registry_vk table by using catalog_name, schema_name, table_name
df = spark.sql(
    f"""
    SELECT *
    FROM ds_training_1.default.schema_registry 
    WHERE catalog_name='{catalog_name}' 
      AND schema_name='{schema_name}' 
      AND table_name='{name}' 
      AND modified_timestamp is not null
      AND schema_change_alert_status ='Pending'
    ORDER BY table_version_timestamp Asc
    Limit 4
    """
)

# Convert the upstream table DataFrame to JSON format
data = df.collect()
json_data = json.dumps(
    [row.asDict() for row in data], 
    default=convert_datetime
)

# Parse the JSON string back into a dictionary
json_dict = json.loads(json_data)

# Access the schema_name from the first element in the list
#print(json_dict[0]["schema_name"])
#print(json_dict)

table_owner = spark.sql(
    f"""
    SELECT DISTINCT created_by 
    FROM ds_training_1.default.schema_registry 
    WHERE catalog_name='{catalog_name}' 
      AND schema_name='{schema_name}' 
      AND table_name='{name}'
    """
)


In [0]:
display(df)

id,catalog_name,schema_name,table_name,schema_version,created_by,modified_by,modified_timestamp,schema_json,change_type,column_name,table_version,table_version_timestamp,status,check_timestamp,schema_change_alert_status,rollback_notification_status
29,ds_training_1,ds_silver,customer_silver_vishal,10,vishal.kokkula@latentviewo365.onmicrosoft.com,vishal.kokkula@latentviewo365.onmicrosoft.com,2024-10-16T10:00:25Z,null,RENAME COLUMN,test_column,12,2024-10-16T10:00:25Z,Inactive,2024-10-16T10:06:49.146321Z,Pending,NA
30,ds_training_1,ds_silver,customer_silver_vishal,11,vishal.kokkula@latentviewo365.onmicrosoft.com,vishal.kokkula@latentviewo365.onmicrosoft.com,2024-10-16T10:02:36Z,null,DROP COLUMNS,date_of_business,13,2024-10-16T10:02:36Z,Inactive,2024-10-16T10:06:49.146324Z,Pending,NA
37,ds_training_1,ds_silver,customer_silver_vishal,13,vishal.kokkula@latentviewo365.onmicrosoft.com,vishal.kokkula@latentviewo365.onmicrosoft.com,2024-10-17T11:08:43Z,"{""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_coloumn_vishal"": ""INT""}",ADD COLUMNS,['test_coloumn_vishal'],18,2024-10-17T11:08:43Z,Inactive,2024-10-17T11:09:43.824667Z,Pending,NA
46,ds_training_1,ds_silver,customer_silver_vishal,14,vishal.kokkula@latentviewo365.onmicrosoft.com,vishal.kokkula@latentviewo365.onmicrosoft.com,2024-10-18T07:59:35Z,"{""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_column_18_10_24"": ""INT""}",RENAME COLUMN,test_column_18_10_24,19,2024-10-18T07:59:35Z,Inactive,2024-10-18T08:31:42.277231Z,Pending,NA


In [0]:
# Function to retrieve email_id using entra_id
def get_email_id_spark(entra_id):
    try:
        # Define the SQL query to get the email_id for a given entra_id
        sql_query = f"""
            SELECT email_id
            FROM ds_training_1.default.email_library
            WHERE entra_id = '{entra_id}'
            LIMIT 1
        """
        
        # Execute the SQL query
        email_id_df = spark.sql(sql_query)
        
        # Collect the result as a Python dictionary
        email_id_row = email_id_df.collect()
        
        # Check if any result was returned
        if email_id_row:
            return email_id_row[0]['email_id']
        else:
            return "N/A"  # Return "N/A" if no match is found
        
    except Exception as e:
        print(f"Failed to retrieve email ID for {entra_id}: {e}")
        return "N/A"

# Example usage of the function
# entra_id = "alagar.kumar@latentviewo365.onmicrosoft.com"
# email_id = get_email_id_spark(entra_id)
# print(f"The email ID for {entra_id} is {email_id}")


In [0]:
import requests
import json



# Function to retrieve SQL query name by query ID
def get_sql_query_name(query_id):
    url = f"https://{databricks_instance}/api/2.0/preview/sql/queries/{query_id}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {databricks_token}"
    }

    # Make the API request to get query details
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        query_data = response.json()
        # Extract the query name if available
        query_name = query_data.get('name', 'N/A')  # Replace 'N/A' with a default if the name is not found
        #print(f"SQL Query Name for query ID {query_id}: {query_name}")
        return query_name
    else:
        print(f"Failed to retrieve query details for query ID {query_id}. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return None




In [0]:
def fetch_created_by(workspace_id, entity_type, entity_id):
    # Check if inputs are valid and not None
    if not entity_id or not entity_type or not workspace_id:
        print(f"Invalid input: workspace_id={workspace_id}, entity_type={entity_type}, entity_id={entity_id}")
        return None
    
    # Construct the SQL command with proper string formatting
    sql_command = (
        f"SELECT DISTINCT created_by "
        f"FROM `system`.access.table_lineage "
        f"WHERE entity_id = '{entity_id}' "  # Ensure entity_id is treated as a string
        f"AND entity_type = '{entity_type}' "  # Ensure entity_type is treated as a string
        f"AND workspace_id = '{workspace_id}'"  # Ensure workspace_id is treated as a string
    )
    
    try:
        # Execute the SQL command
        created_by_df = spark.sql(sql_command)
        
        # Convert the result to a list of dictionaries for easy access
        created_by_data = created_by_df.toPandas().to_dict(orient='records')
        
        # Return the created_by value if available, otherwise return 'N/A'
        if created_by_data and 'created_by' in created_by_data[0]:
            return created_by_data[0]['created_by']
        else:
            return 'N/A'
    except Exception as e:
        print(f"Failed to fetch created_by for {entity_type} with ID {entity_id}: {e}")
        return 'N/A'


In [0]:
def fetch_table_created_by(catalog_name, schema_name, table_name):
    sql_command = (
        f"""select table_owner from `system`.information_schema.tables 
        where table_catalog = '{catalog_name}' 
        and table_schema = '{schema_name}' 
        and table_name = '{table_name}'"""
    )
    try:
        # Execute the SQL command
        created_by_df = spark.sql(sql_command)
        
        # Convert the result to a list of dictionaries for easy access
        created_by_data = created_by_df.toPandas().to_dict(orient='records')
        
        # Return the created_by value if available, otherwise return 'N/A'
        if created_by_data and 'table_owner' in created_by_data[0]:
            return created_by_data[0]['table_owner']
        else:
            return 'N/A'
    except Exception as e:
        print(f"Failed to fetch created_by for table with ID:{table_name},{schema_name},{catalog_name}: {e}")
        return 'N/A'

In [0]:
import requests
import json

# Function to retrieve job name by job ID
def get_job_name(job_id):
    # Construct the API URL for getting job details
    job_url = f"https://{databricks_instance}/api/2.1/jobs/get?job_id={job_id}"
    response = requests.get(job_url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        job_details = response.json()
        
        # Extract the job name from the response
        job_name = job_details.get('settings', {}).get('name')
        
        # Print the job name for debugging
        
        # Return the job name
        return job_name
    else:
        # Handle errors
        print(f"Failed to retrieve job name for Job ID {job_id}: {response.status_code}")
        print(f"Error details: {response.text}")
        return None


In [0]:
def fetch_email_by_member(members):
    email_ids = []
    for member in members:
        sql_command = f"""
            SELECT email_id
            FROM ds_training_1.default.email_library
            WHERE entra_id = '{member}'
            LIMIT 1
        """
        try:
            email_df = spark.sql(sql_command)
            email_data = email_df.toPandas().to_dict(orient='records')
            if email_data and 'email_id' in email_data[0]:
                email_ids.append(email_data[0]['email_id'])
            else:
                email_ids.append('N/A')
        except Exception as e:
            print(f"Failed to fetch email for {member}: {e}")
            email_ids.append('N/A')
    return email_ids

# Function to fetch email using the mapping table
def fetch_email_by_entra_id(entra_id):
    # Perform SQL query on the mapping data table to get the email ID
    sql_command = f"""
            SELECT email_id
            FROM ds_training_1.default.email_library
            WHERE entra_id = '{entra_id}'
            LIMIT 1
        """
    try:
        email_df = spark.sql(sql_command)
        email_data = email_df.toPandas().to_dict(orient='records')
        if email_data and 'email_id' in email_data[0]:
            return email_data[0]['email_id']
        return 'N/A'
    except Exception as e:
        print(f"Failed to fetch email for {entra_id}: {e}")
        return 'N/A'

In [0]:
import requests
import json
import pandas as pd
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()


headers = {
    "Authorization": f"Bearer {databricks_token}",
    "Content-Type": "application/json"
}

# API endpoint for group retrieval
group_url = f"https://{databricks_instance}/api/2.0/preview/scim/v2/Groups"

# Function to fetch version information and append record_id





def fetch_table_permissions_with_record_id(table_info):
    permission_info_with_id = []
    if isinstance(table_info, dict):
        table_info = [table_info]
    for table in table_info:
        if all(key in table for key in ['catalog_name', 'schema_name', 'name']):
            table_path = f"{table['catalog_name']}.{table['schema_name']}.{table['name']}"
            permissions_data = fetch_table_permissions_spark(table_path)  # Fetch actual permissions
            permission_info_with_id.append(permissions_data)
    return permission_info_with_id

# Function to process info types (e.g., TableInfo, JobInfo, NotebookInfo, DashboardInfo, etc.) and add record_id
# Sample function to process entries and add record_id
def process_info_with_record_id(info_list, record_id, entity_type):
    if isinstance(info_list, dict):
        info_list = [info_list]

    for info in info_list:
        # Retrieve 'created_by' using the function with workspace_id, entity_type, and entity_id
        workspace_id = info.get('workspace_id')
        entity_id = str(info.get('job_id') or info.get('notebook_id') or info.get('dashboard_id') or info.get('query_id'))

        # Debugging print
        print(f"Processing record {record_id}: workspace_id={workspace_id}, entity_id={entity_id}")
             
        created_by = fetch_created_by(workspace_id, entity_type, entity_id)
        
        if entity_type == 'Table':
            info['created_by'] = fetch_table_created_by(info.get('catalog_name'), info.get('schema_name'), info.get('name'))
        else:
            info['created_by'] = created_by
        
        info['created_by_email'] = get_email_id_spark(info.get('created_by'))

        # Check if 'created_by' and 'created_by_email' are valid after fetching
        print(f"Record {record_id} created_by: {info['created_by']}, created_by_email: {info['created_by_email']}")

        # Set entity_name based on the entity type
        if entity_type == 'JOB':
            info['entity_name'] = get_job_name(info.get('job_id'))
        elif entity_type == 'DBSQL_QUERY':
            info['entity_name'] = get_sql_query_name(info.get('query_id'))
        else:
            info['entity_name'] = 'N/A'  # Placeholder; replace with actual name if available
        
        info['record_id'] = record_id

    return info_list
# Function to fetch all groups using pagination
def fetch_all_groups():
    start_index = 1
    count = 100  # Number of groups to retrieve per API call
    all_groups = []

    # Fetch all groups using pagination
    while True:
        response = requests.get(group_url, headers=headers, params={"startIndex": start_index, "count": count})
        if response.status_code == 200:
            group_data = response.json().get('Resources', [])
            if not group_data:
                break  # Exit loop if no more groups are found
            all_groups.extend(group_data)
            start_index += count
        else:
            print(f"Failed to retrieve group details: Status Code {response.status_code}")
            break
    return all_groups

# Function to process group details into JSON format
def process_group_details(groups):
    group_list = []
    for group in groups:
        members = [member.get('display', 'N/A') for member in group.get('members', [])]
        email_ids = fetch_email_by_member(members)
        group_info = {
            "Group ID": group.get('id', 'N/A'),
            "Group Name": group.get('displayName', 'N/A'),
            "Members Count": len(group.get('members', [])),
            "Members": [member.get('display', 'N/A') for member in group.get('members', [])],
            "Email IDs": email_ids
        }
        group_list.append(group_info)
    return group_list

# Fetch and process all groups at once
all_groups = fetch_all_groups()
group_data_map = {group["Group Name"]: group for group in process_group_details(all_groups)}

# Function to fetch table permissions using SHOW GRANTS ON
def fetch_table_permissions_spark(table_path):
    if table_path is None or 'None' in table_path:  # Check if table path is valid
        print(f"Skipping SHOW GRANTS ON because table path is invalid: {table_path}")
        return None
    sql_command = f"SHOW GRANTS ON {table_path}"
    try:
        grants_df = spark.sql(sql_command)
        grants_data = grants_df.toPandas().to_dict(orient='records')
        return grants_data  # Return as a list of dictionaries
    except Exception as e:
        print(f"Failed to fetch grants for {table_path}: {e}")
        return None




def process_permissions_and_groups(principal):
    group_info_list = []
    
    if principal is None:
        return group_info_list 
        
        # Check if principal is a group (not containing 'o365.onmicrosoft.com')
    if principal not in group_data_map and "o365.onmicrosoft.com" not in principal:
        group_info = fetch_group_info(principal)
        if group_info:
                # Append the actual group info
            group_info_list.append(group_info)
        
    elif principal in group_data_map:
            # Append data from group_data_map for existing groups
            group_info_list.append(group_data_map[principal])
    
    return group_info_list

# Function to fetch group info for a specific principal
def fetch_group_info(principal):
    if principal in group_data_map:
        return group_data_map[principal]
    return 
    {
        "Group ID": "N/A",
        "Group Name": principal,
        "Members Count": 0,
        "Members": []
    }




In [0]:
# API call to fetch Table Lineage data
lineage_api_url = f"https://{databricks_instance}/api/2.0/lineage-tracking/table-lineage"
payload = {
    "table_name": f"{catalog_name}.{schema_name}.{name}",
    "include_entity_lineage": True
}
upstream_table_path = payload.get('table_name')

response = requests.get(lineage_api_url, headers=headers, data=json.dumps(payload))

if response.status_code == 200:
    lineage_data = response.json()
    print("Table Lineage Data:")
else:
    print(f"Failed to retrieve table lineage: {response.status_code}")
    lineage_data = {}

# Process the lineage data and populate table columns
lineage_rows = []
table_rows = []
permission_rows = [] 
group_rows = []
job_rows = []
notebook_rows = []
dashboard_rows = []
query_rows = []
pipeline_rows = []
model_rows = []
file_rows = []
upstream_schema_changed = json_dict
downstreams = lineage_data.get('downstreams', [])
record_id = 1
current_id = 1

def convert_timestamps_to_str(data):
    for item in data:
        for key, value in item.items():
            if isinstance(value, pd.Timestamp):
                item[key] = value.isoformat()
    return data

for downstream in downstreams:
    # Process TableInfo
    table_info = downstream.get('tableInfo', {})
    record_id = 0
    current_id = 0
    # Ensure table_info is a list
    if not isinstance(table_info, list):
        table_info = [table_info]
    for table in table_info:
        table_data = process_info_with_record_id(table, record_id, 'Table')
        if not isinstance(table_data, dict) and table_data[0]['created_by'] != 'N/A':
            table_rows.append(table_data)
            # Fetch PermissionInfo (already defined elsewhere)
            permission_info_with_id = fetch_table_permissions_with_record_id(table)
            permission=permission_info_with_id[0]
            print(permission)
            # Fetch GroupInfo based on permissions
            group_info_with_id = []
            if permission:

                group_info = []
                if isinstance(permission, list):
                    for perm in permission:
                        principal = perm['Principal']
                        action_type=perm['ActionType']
                        ObjectType=perm['ObjectType']
                        ObjectKey=perm['ObjectKey']
                        if '@latentviewo365.onmicrosoft.com' in principal:  # Check if it's a group (not an email address)
                                # Fetch the email for the user if it's not a group
                                email = fetch_email_by_entra_id(principal)
                                perm['created_by_email_id'] = email
                                print(perm)
                                permission_rows.append(perm)
                        else:
                                print(perm)
                                group_data = process_permissions_and_groups(principal)
                                group_info={
                                    "Previleges":action_type,
                                    "ObjectKey":ObjectKey,
                                    "group_details":group_data
                                }
                                group_rows.append(group_info)

                    

        

        record_id += 1
        current_id += 1

    # Process JobInfo
    job_info = downstream.get('jobInfos', [])
    record_id = 0
    current_id = 0
    for job in job_info:
        job_data = process_info_with_record_id(job, record_id, 'JOB')
        job_rows.append(job_data)
        record_id += 1
        current_id += 1

    # Process NotebookInfo
    notebook_info = downstream.get('notebookInfos', [])
    record_id = 0
    current_id = 0
    for notebook in notebook_info:
        notebook_data = process_info_with_record_id(notebook, record_id, 'NOTEBOOK')
        notebook_rows.append(notebook_data)
        # Increment the record_id and current_id for the next entry
        record_id += 1
        current_id += 1

    # Process DashboardInfo
    dashboard_info = downstream.get('dashboardV3Infos', [])
    record_id = 0
    current_id = 0
    for dashboard in dashboard_info:
        dashboard_data = process_info_with_record_id(dashboard, record_id, 'DASHBOARD_V3')
        dashboard_rows.append(dashboard_data)
        record_id += 1
        current_id += 1

    # Process QueryInfo
    query_info = downstream.get('queryInfos', [])
    record_id = 0
    current_id = 0
    for query in query_info:
        query_data = process_info_with_record_id(query, record_id, 'DBSQL_QUERY')
        query_rows.append(query_data)
        record_id += 1
        current_id += 1

    # Process FileInfo
    file_info = downstream.get("fileInfo", [])
    record_id = 0
    current_id = 0
    for file in file_info:
        file_data = process_info_with_record_id(file, record_id, 'FILE')
        file_rows.append(file_data)
        record_id += 1
        current_id += 1

    # Format all data into a single row (dictionary)
    lineage_row = {
        "Type": "Downstream",
        "source_table": upstream_table_path,
        "table_owner": fetch_email_by_entra_id(table_owner.collect()[0]["created_by"]),
        "upstream_changed_schema_info": upstream_schema_changed,
        "TablesInfo": table_rows,
        "PermissionInfo": permission_rows,
        "GroupInfo": group_rows,
        "JobInfo": job_rows,
        "NotebookInfo": notebook_rows,
        "DashboardInfo": dashboard_rows,
        "QueryInfo": query_rows,
        "PipelineInfo": pipeline_rows,
        "ModelsInfo": model_rows,
        "FileInfo": file_rows
    }

    # Add the formatted row to the list
    lineage_rows.append(lineage_row)
    # Increment the record_id and current_id for the next entry
    record_id += 1
    current_id += 1

# Convert to DataFrame and display
if lineage_rows:
    df_lineage = pd.DataFrame(lineage_rows)
    display(df_lineage)
elif upstream_table_path:
     lineage_row = {
            "Type": "Downstream",
            "source_table": upstream_table_path,
            "table_owner": fetch_email_by_entra_id(table_owner.collect()[0]["created_by"]),
            "upstream_changed_schema_info": upstream_schema_changed,
            "TablesInfo": [],  # Empty as no downstreams were processed
            "PermissionInfo": [],
            "GroupInfo": [],
            "JobInfo": [],
            "NotebookInfo": [],
            "DashboardInfo": [],
            "QueryInfo": [],
            "PipelineInfo": [],
            "ModelsInfo": [],
            "FileInfo": []
        } 
     lineage_rows.append(lineage_row)
     df_lineage = pd.DataFrame(lineage_rows)
     display(df_lineage)   
else:
    print("No lineage data available to display: No downstream entities found")

Table Lineage Data:
Processing record 0: workspace_id=None, entity_id=None
Invalid input: workspace_id=None, entity_type=Table, entity_id=None
Record 0 created_by: N/A, created_by_email: N/A
Processing record 0: workspace_id=2376768479807879, entity_id=4438279648331977
Record 0 created_by: vishal.kokkula@latentviewo365.onmicrosoft.com, created_by_email: vishal.kokkula@latentview.com
Processing record 1: workspace_id=2376768479807879, entity_id=3477860330566096
Record 1 created_by: vishal.kokkula@latentviewo365.onmicrosoft.com, created_by_email: vishal.kokkula@latentview.com
Processing record 2: workspace_id=2376768479807879, entity_id=3477860330566379
Record 2 created_by: vishal.kokkula@latentviewo365.onmicrosoft.com, created_by_email: vishal.kokkula@latentview.com
Processing record 3: workspace_id=2376768479807879, entity_id=3852443579535474
Record 3 created_by: vishal.kokkula@latentviewo365.onmicrosoft.com, created_by_email: vishal.kokkula@latentview.com
Processing record 4: workspac

Type,source_table,table_owner,upstream_changed_schema_info,TablesInfo,PermissionInfo,GroupInfo,JobInfo,NotebookInfo,DashboardInfo,QueryInfo,PipelineInfo,ModelsInfo,FileInfo
Downstream,ds_training_1.ds_silver.customer_silver_vishal,vishal.kokkula@latentview.com,"List(List(ds_training_1, RENAME COLUMN, 2024-10-16T10:06:49.146321, test_column, vishal.kokkula@latentviewo365.onmicrosoft.com, 29, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:00:25, NA, Pending, null, ds_silver, 10, Inactive, customer_silver_vishal, 12, 2024-10-16T10:00:25), List(ds_training_1, DROP COLUMNS, 2024-10-16T10:06:49.146324, date_of_business, vishal.kokkula@latentviewo365.onmicrosoft.com, 30, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:02:36, NA, Pending, null, ds_silver, 11, Inactive, customer_silver_vishal, 13, 2024-10-16T10:02:36), List(ds_training_1, ADD COLUMNS, 2024-10-17T11:09:43.824667, ['test_coloumn_vishal'], vishal.kokkula@latentviewo365.onmicrosoft.com, 37, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-17T11:08:43, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_coloumn_vishal"": ""INT""}, ds_silver, 13, Inactive, customer_silver_vishal, 18, 2024-10-17T11:08:43), List(ds_training_1, RENAME COLUMN, 2024-10-18T08:31:42.277231, test_column_18_10_24, vishal.kokkula@latentviewo365.onmicrosoft.com, 46, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-18T07:59:35, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_column_18_10_24"": ""INT""}, ds_silver, 14, Inactive, customer_silver_vishal, 19, 2024-10-18T07:59:35))","List(List(List(ds_training_1, vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, N/A, 2024-09-22 18:48:26.0, customer_transaction_summary, 0, ds_gold, TABLE)))","List(List(ALL PRIVILEGES, ds_training_1.ds_gold.customer_transaction_summary, TABLE, danduprolu.stuthi@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentview.com))","List(List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(kudumalauday.kumarreddy@latentview.com, danduprolu.stuthi@latentview.com), 485080529618658, Data Engineer, List(kudumalauday.kumarreddy@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(brindavivek.kotha@latentview.com, danduprolu.stuthi@latentview.com), 823552377244743, sample, List(brindavivek.kotha@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1, ALL PRIVILEGES, List()))","List(List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 776564016740257, 2024-09-22 18:48:26.0, 0, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 539953870834493, 2024-09-20 04:45:51.0, 1, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 642259009613085, 2024-09-20 04:36:59.0, 2, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 40710675392154, 2024-09-19 18:39:52.0, 3, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 158830438318304, 2024-09-19 09:41:27.0, 4, 2376768479807879)), List(List(vishal.kokkula

In [0]:
print(type(df_lineage))

<class 'pandas.core.frame.DataFrame'>


In [0]:
df=df_lineage.iloc[0]
df = df.to_frame().T
display(df)


Type,source_table,table_owner,upstream_changed_schema_info,TablesInfo,PermissionInfo,GroupInfo,JobInfo,NotebookInfo,DashboardInfo,QueryInfo,PipelineInfo,ModelsInfo,FileInfo
Downstream,ds_training_1.ds_silver.customer_silver_vishal,vishal.kokkula@latentview.com,"List(List(ds_training_1, RENAME COLUMN, 2024-10-16T10:06:49.146321, test_column, vishal.kokkula@latentviewo365.onmicrosoft.com, 29, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:00:25, NA, Pending, null, ds_silver, 10, Inactive, customer_silver_vishal, 12, 2024-10-16T10:00:25), List(ds_training_1, DROP COLUMNS, 2024-10-16T10:06:49.146324, date_of_business, vishal.kokkula@latentviewo365.onmicrosoft.com, 30, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:02:36, NA, Pending, null, ds_silver, 11, Inactive, customer_silver_vishal, 13, 2024-10-16T10:02:36), List(ds_training_1, ADD COLUMNS, 2024-10-17T11:09:43.824667, ['test_coloumn_vishal'], vishal.kokkula@latentviewo365.onmicrosoft.com, 37, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-17T11:08:43, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_coloumn_vishal"": ""INT""}, ds_silver, 13, Inactive, customer_silver_vishal, 18, 2024-10-17T11:08:43), List(ds_training_1, RENAME COLUMN, 2024-10-18T08:31:42.277231, test_column_18_10_24, vishal.kokkula@latentviewo365.onmicrosoft.com, 46, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-18T07:59:35, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_column_18_10_24"": ""INT""}, ds_silver, 14, Inactive, customer_silver_vishal, 19, 2024-10-18T07:59:35))","List(List(List(ds_training_1, vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, N/A, 2024-09-22 18:48:26.0, customer_transaction_summary, 0, ds_gold, TABLE)))","List(List(ALL PRIVILEGES, ds_training_1.ds_gold.customer_transaction_summary, TABLE, danduprolu.stuthi@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentview.com))","List(List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(kudumalauday.kumarreddy@latentview.com, danduprolu.stuthi@latentview.com), 485080529618658, Data Engineer, List(kudumalauday.kumarreddy@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(brindavivek.kotha@latentview.com, danduprolu.stuthi@latentview.com), 823552377244743, sample, List(brindavivek.kotha@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1, ALL PRIVILEGES, List()))","List(List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 776564016740257, 2024-09-22 18:48:26.0, 0, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 539953870834493, 2024-09-20 04:45:51.0, 1, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 642259009613085, 2024-09-20 04:36:59.0, 2, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 40710675392154, 2024-09-19 18:39:52.0, 3, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 158830438318304, 2024-09-19 09:41:27.0, 4, 2376768479807879)), List(List(vishal.kokkula

In [0]:
# Remove duplicate columns
df = df.loc[:, ~df.columns.duplicated()]
display(df)

Type,source_table,table_owner,upstream_changed_schema_info,TablesInfo,PermissionInfo,GroupInfo,JobInfo,NotebookInfo,DashboardInfo,QueryInfo,PipelineInfo,ModelsInfo,FileInfo
Downstream,ds_training_1.ds_silver.customer_silver_vishal,vishal.kokkula@latentview.com,"List(List(ds_training_1, RENAME COLUMN, 2024-10-16T10:06:49.146321, test_column, vishal.kokkula@latentviewo365.onmicrosoft.com, 29, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:00:25, NA, Pending, null, ds_silver, 10, Inactive, customer_silver_vishal, 12, 2024-10-16T10:00:25), List(ds_training_1, DROP COLUMNS, 2024-10-16T10:06:49.146324, date_of_business, vishal.kokkula@latentviewo365.onmicrosoft.com, 30, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-16T10:02:36, NA, Pending, null, ds_silver, 11, Inactive, customer_silver_vishal, 13, 2024-10-16T10:02:36), List(ds_training_1, ADD COLUMNS, 2024-10-17T11:09:43.824667, ['test_coloumn_vishal'], vishal.kokkula@latentviewo365.onmicrosoft.com, 37, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-17T11:08:43, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_coloumn_vishal"": ""INT""}, ds_silver, 13, Inactive, customer_silver_vishal, 18, 2024-10-17T11:08:43), List(ds_training_1, RENAME COLUMN, 2024-10-18T08:31:42.277231, test_column_18_10_24, vishal.kokkula@latentviewo365.onmicrosoft.com, 46, vishal.kokkula@latentviewo365.onmicrosoft.com, 2024-10-18T07:59:35, NA, Pending, {""customer_id"": ""INT"", ""name"": ""STRING"", ""age"": ""INT"", ""gender"": ""STRING"", ""phone_number"": ""STRING"", ""email"": ""STRING"", ""account_id"": ""INT"", ""account_type"": ""STRING"", ""balance"": ""INT"", ""opened_date"": ""DATE"", ""status"": ""STRING"", ""business_date"": ""STRING"", ""test_column"": ""DATE"", ""test_column_18_10_24"": ""INT""}, ds_silver, 14, Inactive, customer_silver_vishal, 19, 2024-10-18T07:59:35))","List(List(List(ds_training_1, vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, N/A, 2024-09-22 18:48:26.0, customer_transaction_summary, 0, ds_gold, TABLE)))","List(List(ALL PRIVILEGES, ds_training_1.ds_gold.customer_transaction_summary, TABLE, danduprolu.stuthi@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentview.com))","List(List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(kudumalauday.kumarreddy@latentview.com, danduprolu.stuthi@latentview.com), 485080529618658, Data Engineer, List(kudumalauday.kumarreddy@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1.ds_gold.customer_transaction_summary, ALL PRIVILEGES, List(List(List(brindavivek.kotha@latentview.com, danduprolu.stuthi@latentview.com), 823552377244743, sample, List(brindavivek.kotha@latentviewo365.onmicrosoft.com, danduprolu.stuthi@latentviewo365.onmicrosoft.com), 2))), List(ds_training_1, ALL PRIVILEGES, List()))","List(List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 776564016740257, 2024-09-22 18:48:26.0, 0, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 539953870834493, 2024-09-20 04:45:51.0, 1, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 642259009613085, 2024-09-20 04:36:59.0, 2, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 40710675392154, 2024-09-19 18:39:52.0, 3, 2376768479807879)), List(List(vishal.kokkula@latentviewo365.onmicrosoft.com, vishal.kokkula@latentview.com, null, 158830438318304, 2024-09-19 09:41:27.0, 4, 2376768479807879)), List(List(vishal.kokkula

In [0]:
#checking the type of the sample column
print(type(df.at[0, 'GroupInfo']))

<class 'list'>


In [0]:
import pandas as pd
import json

# Create the DataFrame
df_lineage = pd.DataFrame(df)

# Function to convert complex nested structures to JSON strings
def convert_to_json(value):
    return json.dumps(value) if isinstance(value, (list, dict)) else value

# Apply the conversion function to all required columns
columns_to_convert = ['upstream_changed_schema_info', 'TablesInfo', 'PermissionInfo', 'GroupInfo','JobInfo','NotebookInfo','DashboardInfo','QueryInfo','PipelineInfo','ModelsInfo','FileInfo']

for col in columns_to_convert:
    df_lineage[col] = df_lineage[col].apply(convert_to_json)

# Display the converted DataFrame
display(df_lineage)


Type,source_table,table_owner,upstream_changed_schema_info,TablesInfo,PermissionInfo,GroupInfo,JobInfo,NotebookInfo,DashboardInfo,QueryInfo,PipelineInfo,ModelsInfo,FileInfo
Downstream,ds_training_1.ds_silver.customer_silver_vishal,vishal.kokkula@latentview.com,"[{""id"": 29, ""catalog_name"": ""ds_training_1"", ""schema_name"": ""ds_silver"", ""table_name"": ""customer_silver_vishal"", ""schema_version"": 10, ""created_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_timestamp"": ""2024-10-16T10:00:25"", ""schema_json"": null, ""change_type"": ""RENAME COLUMN"", ""column_name"": ""test_column"", ""table_version"": 12, ""table_version_timestamp"": ""2024-10-16T10:00:25"", ""status"": ""Inactive"", ""check_timestamp"": ""2024-10-16T10:06:49.146321"", ""schema_change_alert_status"": ""Pending"", ""rollback_notification_status"": ""NA""}, {""id"": 30, ""catalog_name"": ""ds_training_1"", ""schema_name"": ""ds_silver"", ""table_name"": ""customer_silver_vishal"", ""schema_version"": 11, ""created_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_timestamp"": ""2024-10-16T10:02:36"", ""schema_json"": null, ""change_type"": ""DROP COLUMNS"", ""column_name"": ""date_of_business"", ""table_version"": 13, ""table_version_timestamp"": ""2024-10-16T10:02:36"", ""status"": ""Inactive"", ""check_timestamp"": ""2024-10-16T10:06:49.146324"", ""schema_change_alert_status"": ""Pending"", ""rollback_notification_status"": ""NA""}, {""id"": 37, ""catalog_name"": ""ds_training_1"", ""schema_name"": ""ds_silver"", ""table_name"": ""customer_silver_vishal"", ""schema_version"": 13, ""created_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_timestamp"": ""2024-10-17T11:08:43"", ""schema_json"": ""{\""customer_id\"": \""INT\"", \""name\"": \""STRING\"", \""age\"": \""INT\"", \""gender\"": \""STRING\"", \""phone_number\"": \""STRING\"", \""email\"": \""STRING\"", \""account_id\"": \""INT\"", \""account_type\"": \""STRING\"", \""balance\"": \""INT\"", \""opened_date\"": \""DATE\"", \""status\"": \""STRING\"", \""business_date\"": \""STRING\"", \""test_column\"": \""DATE\"", \""test_coloumn_vishal\"": \""INT\""}"", ""change_type"": ""ADD COLUMNS"", ""column_name"": ""['test_coloumn_vishal']"", ""table_version"": 18, ""table_version_timestamp"": ""2024-10-17T11:08:43"", ""status"": ""Inactive"", ""check_timestamp"": ""2024-10-17T11:09:43.824667"", ""schema_change_alert_status"": ""Pending"", ""rollback_notification_status"": ""NA""}, {""id"": 46, ""catalog_name"": ""ds_training_1"", ""schema_name"": ""ds_silver"", ""table_name"": ""customer_silver_vishal"", ""schema_version"": 14, ""created_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_by"": ""vishal.kokkula@latentviewo365.onmicrosoft.com"", ""modified_timestamp"": ""2024-10-18T07:59:35"", ""schema_json"": ""{\""customer_id\"": \""INT\"", \""name\"": \""STRING\"", \""age\"": \""INT\"", \""gender\"": \""STRING\"", \""phone_number\"": \""STRING\"", \""email\"": \""STRING\"", \""account_id\"": \""INT\"", \""account_type\"": \""STRING\"", \""balance\"": \""INT\"", \""opened_date\"": \""DATE\"", \""status\"": \""STRING\"", \""business_date\"": \""STRING\"", \""test_column\"": \""DATE\"", \""test_column_18_10_24\"": \""INT\""}"", ""change_type"": ""RENAME COLUMN"", ""column_name"": ""test_column_18_10_24"", ""table_version"": 19, ""table_version_timestamp"": ""2024-10-18T07:59:35"", ""status"": ""Inactive"", ""check_timestamp"": ""2024-10-18T08:31:42.277231"", ""schema_change_alert_status"": ""Pending"", ""rollback_notification_status"": ""NA""}]","[[{""name"": ""customer_transaction_summary"", ""catalog_name"": ""ds_training_1"", ""schema_name"": ""ds_gold"", ""table_type"": ""TABLE"", ""lineage_timestamp"": ""2024-09-22 18:48:26.0""

In [0]:
print(df_lineage['source_table'][0])

ds_training_1.ds_silver.customer_silver_vishal


In [0]:
# Convert Pandas DataFrame to Spark DataFrame
df_spark = spark.createDataFrame(df_lineage)

# Rename columns to make them unique
df = df.rename(columns=lambda x: f"{x}_{df.columns.tolist().count(x)}" if df.columns.tolist().count(x) > 1 else x)

# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df)

# Write the Spark DataFrame to the Delta table
df_spark.write.format("delta").mode("append").saveAsTable("ds_training_1.default.downstream_audit")


In [0]:
id = spark.sql(f"""
    SELECT ID 
    FROM ds_training_1.default.downstream_audit
    order by ID desc
    limit 1
""")

In [0]:
print(type(id))

<class 'pyspark.sql.connect.dataframe.DataFrame'>


In [0]:
id=id.collect()[0][0]
print(id)
print(type(id))

12
<class 'int'>


In [0]:
try:
    print("Calling email alert notebook")
    # Setting a timeout value (e.g., 300 seconds)
    audit_notebook_result = dbutils.notebook.run(
        "Email_alert", timeout_seconds=300, arguments={"id": str(id)}
    )
    print("Email alert notebook executed successfully.")
except Exception as e:
    print(f"Email alert notebook failed: {e}")

    

Calling email alert notebook
Email alert notebook executed successfully.
